In [ ]:
import numpy as np
from scipy import stats

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
import pprint

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pymongo
import os, sys
# sys.path = [root_dir] + sys.path

In [ ]:
from bokeh.io import push_notebook, show, output_notebook, export_svgs, export_png, save
from bokeh.layouts import column, row, gridplot
from bokeh.plotting import figure
from bokeh.models import Band, Span, Range1d
from bokeh.models.sources import ColumnDataSource
from bokeh.models.formatters import NumeralTickFormatter
from bokeh.palettes import all_palettes, magma, Set3
from bokeh.models import Range1d
from bokeh.models import FixedTicker
output_notebook()

In [ ]:
def ewma(x, alpha):
    '''
    Returns the exponentially weighted moving average of x.

    Parameters:
    -----------
    x : array-like
    alpha : float {0 <= alpha <= 1}

    Returns:
    --------
    ewma : numpy array
        the exponentially weighted moving average
    '''
    # coerce x to an array
    x = np.array(x)
    n = x.size
    # create an initial weight matrix of (1-alpha), and a matrix of powers
    # to raise the weights by
    w0 = np.ones(shape=(n,n)) * (1-alpha) 
    p = np.vstack([np.arange(i,i-n,-1) for i in range(n)])
    # create the weight matrix
    w = np.tril(w0**p,0)
    # calculate the ewma
    return np.dot(w, x[::np.newaxis]) / w.sum(axis=1)

In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = np.array(data)
    n = a.shape[0]
    m, se = a.mean(axis=0), stats.sem(a, axis=0)
    h = se * stats.t._ppf((1+confidence)/2., n-1)
    return m, h

In [ ]:
def get_stats(y_key):
    means = {}
    stds = {}
    nums = {}
    for key in tqdm_notebook(keys, desc="params", leave=False):
        runs = data[key][:limit]
#         print(runs)
        interps = []

        for run in tqdm_notebook(runs, desc="runs", leave=False):
            
            if "{}_T".format(y_key) not in run:
                print("Field: {} not in run for {}".format(y_key, key))
                continue
            
            xs = run["{}_T".format(y_key)]
            ys = run[y_key]
            
#             xs = [a["py/tuple"][0] for a in run]
#             ys = [a["py/tuple"][1] for a in run]
            
            # Check they are long enough
            if xs[-1] < t_needed:
                if print_not_long_enough:
                    print("Run not long enough: {} for key: {}".format(xs[-1], key))
                continue

            # Align them
#             diffs = len(xs) - len(ys)
#             xs = xs[diffs:]
            y_interp = np.interp(x_interp, xs, ys)

            y_interp_smoothed = ewma(y_interp, smoother)
#             y_interp_smoothed = y_interp
    
            interps.append(y_interp_smoothed)
        
        if interps == []:
            continue

        joined_array = np.array(interps)
        if confidence_interval:
            mean, std = mean_confidence_interval(joined_array, confidence=confidence_interval)
        else:
            mean = np.mean(joined_array, axis=0)
            std = np.std(joined_array, axis=0)
        means[key] = mean
        stds[key] = std
        nums[key] = len(interps)
    return means, stds, nums

In [ ]:
def plot(means, stds, nums, x_label, y_label, t_max, indivs=False, title=''):
    
    x_vals_interp = x_interp
          
    keys =  means.keys()
    p = figure(plot_width=1400, plot_height=800, x_range=[0, t_max], title=title)

    y_min = 0
    y_max = 1

    num_lines = len(keys)
    if num_lines < 3:
        magma_cols = ["red", "green"][:num_lines]
    else:
        magma_cols = Set3[num_lines]
    colors = {}
    for key, col in zip(keys, magma_cols):
        colors[key] = col
        
    for key in keys:
        if 'checkpoints' in key and 'ff' in key:
            color = '#fb8072'
        elif 'checkpoints' in key and 'rnn' in key:
            color = '#bebada'
        elif 'ff' in key:
            color = '#ffffb3'
        else:
            color = '#8dd3c7'
        
        name = key

        p.line(x_vals_interp, means[key], color=color, legend=name + " [" + str(nums[key]) + "]", line_width=5)

        xs = list(x_vals_interp) + list(reversed(x_vals_interp))

        mm = means[key]
        ss = stds[key]
        ys = np.concatenate([mm - ss, np.flip(mm + ss, axis=0)])
        lls = p.patch(xs, ys, color=color, alpha=0.2)

        y_max = max(y_max, max(means[key]))
        y_min = min(y_min, min(means[key]))
        
        if indivs:
            for run in data[key][:limit]:
                if "{}_T".format(y_label) not in run:
                    continue

                xs = run["{}_T".format(y_label)]
                ys = run[y_label]

                p.line(xs, ys, color=color, line_width=1, alpha=0.4)

                y_max = max(y_max, max(ys))
                y_min = min(y_min, min(ys))
        
    p.y_range = Range1d(y_min * 1.1, y_max * 1.1)

    p.grid.grid_line_width=2
    p.grid.minor_grid_line_width=2
    p.grid.grid_line_color=(1,1,1,0.1)

    p.grid.minor_grid_line_color=(1,1,1,0.1)
    p.xgrid.minor_grid_line_color=None

    p.yaxis.formatter = NumeralTickFormatter(format="0.0a")
    p.yaxis[0].ticker.desired_num_ticks = 5
    p.yaxis[0].ticker.num_minor_ticks = 2

    p.xaxis.formatter = NumeralTickFormatter(format="0.0a")

    p.legend.location = (0,0)

    p.xaxis.axis_label= x_label
    p.yaxis.axis_label= y_label

#     p.xaxis.axis_label_standoff=-20
#     p.yaxis.axis_label_standoff=-30

    p.xaxis.axis_label_text_font_style="normal"
    p.yaxis.axis_label_text_font_style="normal"

    p.xaxis.axis_label_text_font_size="26pt"
    p.yaxis.axis_label_text_font_size="26pt"

    p.xaxis.major_label_text_font_size="24pt"
    p.yaxis.major_label_text_font_size="24pt"

    # p.legend.label_text_font_size="16pt"
    p.legend.visible=True
    p.legend.location="top_right"
    # p.legend.orientation="horizontal"
    # p.legend.glyph_width=60
    # p.legend.glyph_height=100

    p.title.text_font_size="24pt"

#    p.h_symmetry = False
    p.min_border_right = 70
    p.min_border_left = 0
    p.min_border_bottom = 0
#     p.yaxis.axis_label_text_line_height = 1
#     p.yaxis.bounds = (400,400)
#     p.yaxis.bounds
#     p.yaxis.axis_label_text_baseline="top"
#     p.yaxis.axis_label_text_align="center"
    p.min_border_top = 30
#     p.border_fill_color = (1,1,1,0.1)

    p.axis.axis_line_width = 3
    p.axis.major_tick_line_width = 5
    p.axis.major_tick_in = 10
    p.axis.major_tick_out = 5

    show(p, notebook_handle=True)
    # save(p, filename="{}.png".format(title))
    # export_png(plot, filename="{}.png".format(title))

In [ ]:
label = "gfootball_iql_simple115_22_4"

In [ ]:
import json

# Load from sacred, dictionaries for config and info. Make sure the label is correct.
exps = []

subfolders = [f.path for f in os.scandir('/Users/conglu/Docker Repos/pymarl/cong_scripts/sacred') if f.is_dir()]

for sf in subfolders:
    with open('{0}/config.json'.format(sf)) as json_file:
        config = json.load(json_file)
    with open('{0}/info.json'.format(sf)) as json_file:
        info = json.load(json_file)
    exps.append({'info' : info, 'config' : config})

In [ ]:
# Keys
info_keys = set()
for exp in exps:
    exp_keys = exp["info"].keys()
    for key in exp_keys:
        info_keys.add(key)
print("Keys in info:")
pprint.pprint(list(filter(lambda x: not x.endswith("_T"), sorted(info_keys))))

In [ ]:
params = ["name"]

t_max = 20 * 1000 * 1000
t_needed = t_max - 18 * 1000 * 1000

limit = 50
x_key = "T env" # x-axis label
x_interp = np.linspace(0, t_max, 2001)

confidence_interval = False
smoother = 0.25

In [ ]:
for scenario in ["academy_empty_goal",
                    "academy_empty_goal_close",
                  "academy_run_to_score",
                  "academy_run_to_score_with_keeper",
                  "academy_pass_and_shoot_with_keeper",
                  "academy_run_pass_and_shoot_with_keeper",
                  "academy_3_vs_1_with_keeper",
                 "academy_counterattack_easy"]:
    keys = set()
    data = {}
    for exp in exps:
        exp_config = exp["config"]
        if not exp_config['env_args']['scenario'] == scenario:
            continue
        if exp["info"] == {}:
            continue
        exp_info = exp["info"]
    #     if exp_info == {}:
    #         continue
        params_str = "__".join(["{}-{}".format(param, exp_config[param]) for param in params])
        keys.add(params_str)
        if params_str not in data:
            data[params_str] = [exp_info]
        else:
            data[params_str].append(exp_info)
    
    print_not_long_enough = False

    for y_key in tqdm_notebook(['ep_length_mean',
 'episode',
 'epsilon',
 'grad_norm',
 'loss',
 'q_taken_mean',
 'return_mean',
 'return_std',
 'score_reward_mean',
 'target_mean',
 'td_error_abs',
 'test_ep_length_mean',
 'test_return_mean',
 'test_return_std',
 'test_score_reward_mean'], desc="keys", leave=False):
        x_key = "T env"
        m, s, n = get_stats(y_key)
        print(n)
        plot(m, s, n, x_key, y_key, t_max, indivs=True, title=scenario)